# LIGHTNINGMODULE
### 본 문서는 PyTorch Lightning의 [공식 가이드](https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html#minimal-example)의 한글 번역본입니다. (옮긴이 [dnap512](https://github.com/dnap512), 21.7.14)

`LightningModule`은 5가지 부분의 PyTorch 코드로 구성됩니다.

- Computations (init)
- Train loop (training_step)
- Validation loop (validation_step)
- Test loop (test_step)
- Optimizers (configure_optimizers)


[가이드 동영상](https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/pl_docs/pl_mod_vid.m4v)을 참고하세요!

몇 가지 사항에 유의하세요.

1. 동일한 코드입니다.
2. PyTorch 코드는 추상화되지 않고 구성됩니다.
3. LightningModule에 없는 다른 모든 코드는 트레이너에 의해 자동화되었습니다.

```python
net = Net()
trainer = Trainer()
trainer.fit(net)
```
4. .cuda()나 .to() 호출은 없습니다. Lightning이 알아서 해줍니다!

```python
# don't do in lightning
x = torch.Tensor(2, 3)
x = x.cuda()
x = x.to(device)

# do this instead
x = x  # leave it alone!

# or to init a new tensor
new_x = torch.Tensor(2, 3)
new_x = new_x.type_as(x)
```

5. Lightning은 여러분을 위해 기본값으로 distrbuted sampler를 다룹니다.

```python
# Don't do in Lightning...
data = MNIST(...)
sampler = DistributedSampler(data)
DataLoader(data, sampler=sampler)

# do this instead
data = MNIST(...)
DataLoader(data)
```

6. `LightningModule`은 [`torch.nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module) 이긴 하지만 기능이 추가된 것입니다. 그대로 사용하세요!

따라서 Lightning을 사용하려면 약 30분 정도 소요되는 코드를 구성하기만 하면 됩니다(진짜로요).

---

## Minimal Example (최소 예제?)

다음은 필요한 최소한의 메서드들만 구현된 예제입니다.


다음을 수행하여 훈련할 수 있습니다.

In [ ]:
import os
import pytorch_lightning as pl

from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader


class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 10)
    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)
    
train_loader = DataLoader(MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()))
trainer = pl.Trainer()
model = LitModel()

trainer.fit(model, train_loader)

LightningModule에는 많은 편리한 방법이 있지만 알아야 할 핵심 방법은 다음과 같습니다.

|이름|설명|
|:---|:---|
|init|여기서 연산을 정의합니다|
|forward|추론만을 위해 작성합니다.(training_step과는 별도로 분리)|
|training_step|전체 학습 루프|
|validation_step|전체 검증 루프|
|test_step|전체 테스트 루프|
|configure_optimizers|옵티마이저와 LR scheduler들을 정의합니다|

## Training

### Training loop

학습 루프를 추가하기 위해서 `training_step` method를 추가하세요.


In [ ]:
class LitClassifier(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

내부에서 Lightning은 다음을 수행합니다(pseudocode).


In [ ]:
# put model in train mode
model.train()
torch.set_grad_enabled(True)

losses = []
for batch in train_dataloader:
    # forward
    loss = training_step(batch)
    losses.append(loss.detach())

    # clear gradients
    optimizer.zero_grad()

    # backward
    loss.backward()

    # update parameters
    optimizer.step()

#### Training epoch-level metrics

만약 여러분이 epoch 단위로 meric을 계산하고 로깅하고 싶다면, `.log()` 메소드를 사용하세요.

In [ ]:
def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)

    # logs metrics for each training_step,
    # and the average across the epoch, to the progress bar and logger
    self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return loss

`.log` 오브젝트는 전체 Epochs에서 요청된 Metric을 자동으로 Reduce합니다. 내부에서 수행하는 작업의 Psuedocode는 다음과 같습니다.

In [ ]:
outs = []
for batch in train_dataloader:
    # forward
    out = training_step(val_batch)

    # clear gradients
    optimizer.zero_grad()

    # backward
    loss.backward()

    # update parameters
    optimizer.step()

epoch_metric = torch.mean(torch.stack([x['train_loss'] for x in outs]))

#### Train epoch-level operations

각 training_step의 모든 출력으로 작업을 수행해야 하는 경우 training_epoch_end를 직접 재정의하세요.

In [ ]:
def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    preds = ...
    return {'loss': loss, 'other_stuff': preds}

def training_epoch_end(self, training_step_outputs):
    for pred in training_step_outputs:
       # do something

해당하는 Psuedocode는 다음과 같습니다.

In [ ]:
outs = []
for batch in train_dataloader:
    # forward
    out = training_step(val_batch)

    # clear gradients
    optimizer.zero_grad()

    # backward
    loss.backward()

    # update parameters
    optimizer.step()

training_epoch_end(outs)

#### Training with DataParallel

GPU에서 각 배치의 데이터를 분할하는 `accelerator`를 사용하여 훈련할 때, 처리를 위해 마스터 GPU에서 데이터를 집계해야 할 수도 있습니다(dp 또는 ddp2).

이 경우 `training_step_end` 메소드를 구현하세요.

In [ ]:
def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    pred = ...
    return {'loss': loss, 'pred': pred}

def training_step_end(self, batch_parts):
    # predictions from each GPU
    predictions = batch_parts['pred']
    # losses from each GPU
    losses = batch_parts['loss']

    gpu_0_prediction = predictions[0]
    gpu_1_prediction = predictions[1]

    # do something with both outputs
    return (losses[0] + losses[1]) / 2

def training_epoch_end(self, training_step_outputs):
    for out in training_step_outputs:
       # do something with preds

Lightning 내부에서 수행되는 전체 Psuedocode는 다음과 같습니다.

In [ ]:
outs = []
for train_batch in train_dataloader:
    batches = split_batch(train_batch)
    dp_outs = []
    for sub_batch in batches:
        # 1
        dp_out = training_step(sub_batch)
        dp_outs.append(dp_out)

    # 2
    out = training_step_end(dp_outs)
    outs.append(out)

# do something with the outputs for all batches
# 3
training_epoch_end(outs)

Validation은 Training과 기본 구조가 같기 때문에 생략합니다.

---

### Test loop

테스트 루프를 추가하는 과정은 검증 루프를 추가하는 과정과 동일합니다. 자세한 내용은 위의 섹션을 참조하세요.

유일한 차이점은 테스트 루프는 `.test()`가 직접 사용될 때만 호출된다는 것입니다.

In [ ]:
model = Model()
trainer = Trainer()
trainer.fit()

# automatically loads the best weights for you
trainer.test(model)

`test()`를 호출하는 두 가지 방법이 있습니다.

In [ ]:
# call after training
trainer = Trainer()
trainer.fit(model)

# automatically auto-loads the best weights
trainer.test(dataloaders=test_dataloader)

# or call with pretrained model
model = MyLightningModule.load_from_checkpoint(PATH)
trainer = Trainer()
trainer.test(model, dataloaders=test_dataloader)

---

## Inference

연구 목적으로 LightningModule은 시스템으로서 가장 잘 구성될 수 있습니다.

In [1]:
import pytorch_lightning as pl
import torch
from torch import nn

class Autoencoder(pl.LightningModule):

    def __init__(self, latent_dim=2):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 256), nn.ReLU(), nn.Linear(256, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 256), nn.ReLU(), nn.Linear(256, 28 * 28))

    def training_step(self, batch, batch_idx):
        x, _ = batch

        # encode
        x = x.view(x.size(0), -1)
        z = self.encoder(x)

        # decode
        recons = self.decoder(z)

        # reconstruction
        reconstruction_loss = nn.functional.mse_loss(recons, x)
        return reconstruction_loss

    def validation_step(self, batch, batch_idx):
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        recons = self.decoder(z)
        reconstruction_loss = nn.functional.mse_loss(recons, x)
        self.log('val_reconstruction', reconstruction_loss)

    def predict_step(self, batch, batch_idx, dataloader_idx):
        x, _ = batch

        # encode
        # for predictions, we could return the embedding or the reconstruction or both based on our need.
        x = x.view(x.size(0), -1)
        return self.encoder(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0002)

그리고 아래처럼 모델을 훈련할 수 있습니다.

In [ ]:
autoencoder = Autoencoder()
trainer = pl.Trainer(gpus=1)
trainer.fit(autoencoder, train_dataloader, val_dataloader)

이 간단한 모델은 다음과 같은 예제를 생성합니다(인코더와 디코더가 너무 약하네요).

![이미지](https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/pl_docs/ae_docs.png)

위의 방법은 Lightning 인터페이스의 일부입니다.

- training_step
- validation_step
- test_step
- predict_step
- configure_optimizers

다음 예제에는 train 루프와 val 루프는 정확히 동일합니다. 물론 이 코드를 재사용할 수 있습니다.

In [ ]:
class Autoencoder(pl.LightningModule):
    def __init__(self, latent_dim=2):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 256), nn.ReLU(), nn.Linear(256, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 256), nn.ReLU(), nn.Linear(256, 28 * 28))

    def training_step(self, batch, batch_idx):
        loss = self.shared_step(batch)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.shared_step(batch)
        self.log('val_loss', loss)

    def shared_step(self, batch):
        x, _ = batch

        # encode
        x = x.view(x.size(0), -1)
        z = self.encoder(x)

        # decode
        recons = self.decoder(z)

        # loss
        return nn.functional.mse_loss(recons, x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0002)

우리는 모든 루프에서 사용할 수 있는 `shared_step`이라는 새 메서드를 만듭니다. 이 메소드 이름은 임의적이며 예약되지 않습니다.

### Inference in Research

시스템에서 추론을 수행하려는 경우 LightningModule에 forward 메서드를 추가할 수 있습니다.

> <span style='color:blue'>NOTE:</span> forward를 사용할 때 `eval()`을 호출하고 `no_grad()` context manager를 사용해야 합니다.

In [ ]:
class Autoencoder(pl.LightningModule):

    def forward(self, x):
        return self.decoder(x)

model = Autoencoder()
model.eval()
with torch.no_grad():
    reconstruction = model(embedding)

`forward`를 추가하는 것의 장점은 복잡한 시스템에서 텍스트 생성과 같은 훨씬 더 복잡한 추론 절차를 수행할 수 있다는 것입니다.

In [ ]:
class Seq2Seq(pl.LightningModule):

    def forward(self, x):
        embeddings = self(x)
        hidden_states = self.encoder(embeddings)
        for h in hidden_states:
            # decode
            ...
        return decoded

추론을 확장하려는 경우에는 [`predict_step()`](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.core.lightning.html#pytorch_lightning.core.lightning.LightningModule.predict_step)을 사용해야 합니다.

In [ ]:
class Autoencoder(pl.LightningModule):

    def forward(self, x):
        return self.decoder(x)

    def predict_step(self, batch, batch_idx, dataloader_idx = None)
        # this calls forward
        return self(batch)

data_module = ...
model = Autoencoder()
trainer = Trainer(gpus=2)
trainer.predict(model, data_module)

### Inference in Production

프로덕션과 같은 경우 LightningModule 내에서 다른 모델을 반복할 수 있습니다.

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.metrics import functional as FM

class ClassificationTask(pl.LightningModule):

    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {'val_acc': acc, 'val_loss': loss}
        self.log_dict(metrics)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {'test_acc': acc, 'test_loss': loss}
        self.log_dict(metrics)
        return metrics

    def _shared_eval_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        acc = FM.accuracy(y_hat, y)
        return loss, acc

    def predict_step(self, batch, batch_idx, dataloader_idx):
        x, y = batch
        y_hat = self.model(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.02)

그런 다음 이 작업에 적합하도록 임의의 모델을 전달합니다.

In [ ]:
for model in [resnet50(), vgg16(), BidirectionalRNN()]:
    task = ClassificationTask(model)

    trainer = Trainer(gpus=2)
    trainer.fit(task, train_dataloader, val_dataloader)

작업들은 GAN 학습, Self-supervised 또는 RL 구현과 같이 복잡한 것도 가능합니다.

In [ ]:
class GANTask(pl.LightningModule):

    def __init__(self, generator, discriminator):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        ...

다음 상황처럼 사용하면 모델을 작업에서 분리할 수 있으므로 LightningModule에 보관할 필요 없이 프로덕션에서 사용할 수 있습니다.

- onnx로 내보낼 수 있습니다.
- 또는 Jit을 사용하여 추적합니다.
- 또는 파이썬 런타임에서 실행하십시오.

In [ ]:
task = ClassificationTask(model)

trainer = Trainer(gpus=2)
trainer.fit(task, train_dataloader, val_dataloader)

# use model after training or load weights and drop into the production system
model.eval()
y_hat = model(x)

LightningModule API에 대한 더 자세한 내용은 [여기](https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html#lightningmodule-api)를 참고하세요!